## Trypout 
https://www.linkedin.com/pulse/self-reflecting-ai-agents-using-langchain-vijaykumar-kartha-yyqtc/

In [24]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_v2"]="false"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [1]:
os.environ["OPENAI_API_KEY1"]

NameError: name 'os' is not defined

In [19]:
# pip install langgraph

In [3]:
from langgraph.graph import END, MessageGraph


In [4]:
## Generation Node
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from typing import List, Sequence

async def generation_node(state: Sequence[BaseMessage]):
	return await generate.ainvoke({"messages": state})

In [5]:
## Reflection Node

async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)

In [6]:
## Self reflection loop graph
builder = MessageGraph()
builder.add_node("generate",generation_node)
builder.add_node("reflect",reflection_node)
builder.set_entry_point("generate")

from typing import List, Sequence

def should_continue(state: List[BaseMessage]):
	if len(state) > 6:
		return END
	return "reflect"


builder.add_conditional_edges("generate",should_continue)
builder.add_edge("reflect","generate")

In [7]:
graph = builder.compile()

In [15]:
# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_mermaid_png()))

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
generation_prompt = ChatPromptTemplate.from_messages(
	[
		(
			"system", "You are an essay assistant tasked with writing excellent 5-paragraph essays."
			"Generate the best essay possible for the user's request."
			" If the user provides critique, respond with a revised version of your previous attempts.",
		),
		MessagesPlaceholder(variable_name="messages"),
	]
) 

In [26]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [34]:
generate = generation_prompt|llm

In [37]:
generate

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [30]:
generate.invoke({"messages": []})

AIMessage(content='I need an essay on the importance of education.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 48, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ad7106d9-a85a-4d51-98cb-ce7ce4d72c44-0', usage_metadata={'input_tokens': 48, 'output_tokens': 11, 'total_tokens': 59, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [33]:
## Reflect Chain
reflection_prompt = ChatPromptTemplate.from_messages(
	[
		(
			"system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
    ),
	  MessagesPlaceholder(variable_name="messages"),
	]
)

In [35]:
reflect = reflection_prompt | llm

In [38]:
reflect.invoke({"messages": []})

APIConnectionError: Connection error.

In [39]:
###Invoking the Self-Reflecting AI Agent
async for event in graph.astream(
	[
	HumanMessage(content="Generate an essay on the topicality of The Little Prince and its message in modern life"
		)
	],
	):
	print(event)
	print("________")

APIConnectionError: Connection error.